In [1]:
import pandas as pd
import numpy as np
import torch
from config import basic_conf as conf
from libs import ModelManager as mm
from config.constants import HyperParamKey, LoadingKey

import logging

import matplotlib.pyplot as plt
from importlib import reload

%matplotlib inline

In [2]:
conf.init_logger(logging.INFO, logfile=None)
mgr = mm.ModelManager(mode='notebook')

[2018-10-28 13:04:38] [INFO] Initializing Model Manager, version 0.4.0 ...
[2018-10-28 13:04:38] [INFO] 
=== Models Available ===
BagOfWords
NLIRNN
NLICNN
[2018-10-28 13:04:38] [INFO] 
=== Loaders Available ===
IMDB
SNLI
MNLI
[2018-10-28 13:04:38] [INFO] 
*********** Model Manager Details ***********
-- self.hparams.num_epochs = 10
-- self.hparams.lr = 0.01
-- self.hparams.voc_size = 100000
-- self.hparams.train_loop_check_freq = 10
-- self.hparams.dropout_rnn = 0.5
-- self.hparams.dropout_fc = 0.5
-- self.hparams.batch_size = 256
-- self.hparams.fc_hidden_size = 100
-- self.hparams.rnn_hidden_size = 50
-- self.hparams.cnn_hidden_size = 100
-- self.hparams.cnn_kernal_size = 3
-- self.hparams.rnn_num_layers = 1
-- self.hparams.check_early_stop = True
-- self.hparams.es_look_back = 50
-- self.hparams.no_imp_look_back = 25
-- self.hparams.decay_lr_no_improv = 0.5
-- self.hparams.es_req_prog = 0.0
-- self.hparams.optim_enc = <class 'torch.optim.adam.Adam'>
-- self.hparams.optim_dec = <clas

### Loading the RNN Model and Validating its Acc

In [3]:
mgr.load_data(mm.loaderRegister.SNLI)

[2018-10-28 12:56:25] [INFO] Loading data using SNLI ...
[2018-10-28 12:56:25] [INFO] loading raw training data set ...
[2018-10-28 12:56:25] [INFO] loading raw training data set ...
[2018-10-28 12:56:25] [INFO] loading pre-trained word vectors, building vocab ...
[2018-10-28 12:56:38] [INFO] converting training set to index ...
[2018-10-28 12:56:39] [INFO] converting val set to index ...
[2018-10-28 12:56:39] [INFO] piping data into pytorch DataLoaders ...


In [4]:
hparams={
    HyperParamKey.LR: 0.01,
    HyperParamKey.SCHEDULER_GAMMA: 0.09,
    HyperParamKey.RNN_HIDDEN_SIZE: 100,
    HyperParamKey.DROPOUT_FC: 0.25,
    HyperParamKey.DROPOUT_RNN: 0.25,
}
mgr.hparams.update(hparams)
mgr.new_model(mm.modelRegister.NLIRNN, label='modlrinv100decay95rhs100drop25')
mgr.load_model(which_model=LoadingKey.LOAD_BEST)

[2018-10-28 12:56:39] [INFO] 
*********** Model: modlrinv100decay95rhs100drop25 Details ***********
-- self.label = modlrinv100decay95rhs100drop25
-- self.hparams.num_epochs = 10
-- self.hparams.lr = 0.01
-- self.hparams.voc_size = 100000
-- self.hparams.train_loop_check_freq = 10
-- self.hparams.dropout_rnn = 0.25
-- self.hparams.dropout_fc = 0.25
-- self.hparams.batch_size = 256
-- self.hparams.fc_hidden_size = 100
-- self.hparams.rnn_hidden_size = 100
-- self.hparams.cnn_hidden_size = 100
-- self.hparams.cnn_kernal_size = 3
-- self.hparams.rnn_num_layers = 1
-- self.hparams.check_early_stop = True
-- self.hparams.es_look_back = 50
-- self.hparams.no_imp_look_back = 25
-- self.hparams.decay_lr_no_improv = 0.5
-- self.hparams.es_req_prog = 0.0
-- self.hparams.optim_enc = <class 'torch.optim.adam.Adam'>
-- self.hparams.optim_dec = <class 'torch.optim.adam.Adam'>
-- self.hparams.scheduler = <class 'torch.optim.lr_scheduler.ExponentialLR'>
-- self.hparams.scheduler_gamma = 0.09
-- self.h

In [5]:
mgr.model.eval_model(mgr.dataloader.loaders['val'])

(70.4, 3.5265949964523315)

### Evaluating the Acc on a MNLI Set

In [6]:
rnn_res = {}
for genre in conf.GENRE_LIST:
    mgr.load_data(mm.loaderRegister.MNLI, genre=genre)
    acc = mgr.model.eval_model(mgr.dataloader.loaders['val'])[0]
    rnn_res[genre] = acc
    
rnn_res

[2018-10-28 12:57:28] [INFO] Loading data using MNLI ...
[2018-10-28 12:57:28] [INFO] loading raw training data set ...
[2018-10-28 12:57:28] [INFO] loading raw training data set ...
[2018-10-28 12:57:28] [INFO] loading pre-trained word vectors, building vocab ...
[2018-10-28 12:57:41] [INFO] converting training set to index ...
[2018-10-28 12:57:42] [INFO] converting val set to index ...
[2018-10-28 12:57:42] [INFO] piping data into pytorch DataLoaders ...
[2018-10-28 12:57:42] [INFO] Loading data using MNLI ...
[2018-10-28 12:57:42] [INFO] loading raw training data set ...
[2018-10-28 12:57:42] [INFO] loading raw training data set ...
[2018-10-28 12:57:42] [INFO] loading pre-trained word vectors, building vocab ...
[2018-10-28 12:57:55] [INFO] converting training set to index ...
[2018-10-28 12:57:55] [INFO] converting val set to index ...
[2018-10-28 12:57:55] [INFO] piping data into pytorch DataLoaders ...
[2018-10-28 12:57:56] [INFO] Loading data using MNLI ...
[2018-10-28 12:57:5

{'fiction': 44.52261306532663,
 'telephone': 46.865671641791046,
 'slate': 44.81037924151697,
 'government': 44.98031496062992,
 'travel': 45.824847250509166}

### CNN

In [7]:
# resetting
mgr = mm.ModelManager(mode='notebook')
mgr.load_data(mm.loaderRegister.SNLI)

[2018-10-28 13:06:31] [INFO] Initializing Model Manager, version 0.4.0 ...
[2018-10-28 13:06:31] [INFO] 
=== Models Available ===
BagOfWords
NLIRNN
NLICNN
[2018-10-28 13:06:31] [INFO] 
=== Loaders Available ===
IMDB
SNLI
MNLI
[2018-10-28 13:06:31] [INFO] 
*********** Model Manager Details ***********
-- self.hparams.num_epochs = 10
-- self.hparams.lr = 0.01
-- self.hparams.voc_size = 100000
-- self.hparams.train_loop_check_freq = 10
-- self.hparams.dropout_rnn = 0.5
-- self.hparams.dropout_fc = 0.5
-- self.hparams.batch_size = 256
-- self.hparams.fc_hidden_size = 100
-- self.hparams.rnn_hidden_size = 50
-- self.hparams.cnn_hidden_size = 100
-- self.hparams.cnn_kernal_size = 3
-- self.hparams.rnn_num_layers = 1
-- self.hparams.check_early_stop = True
-- self.hparams.es_look_back = 50
-- self.hparams.no_imp_look_back = 25
-- self.hparams.decay_lr_no_improv = 0.5
-- self.hparams.es_req_prog = 0.0
-- self.hparams.optim_enc = <class 'torch.optim.adam.Adam'>
-- self.hparams.optim_dec = <clas

In [8]:
hparams={
    HyperParamKey.LR: 0.01,
    HyperParamKey.CNN_HIDDEN_SIZE: 200,
    HyperParamKey.CNN_KERNAL_SIZE: 3
}
mgr.hparams.update(hparams)
mgr.new_model(mm.modelRegister.NLICNN, label='cnnlrinv1000hs200kern3')
mgr.load_model(which_model=LoadingKey.LOAD_BEST)

[2018-10-28 13:06:45] [INFO] 
*********** Model: cnnlrinv1000hs200kern3 Details ***********
-- self.label = cnnlrinv1000hs200kern3
-- self.hparams.num_epochs = 10
-- self.hparams.lr = 0.01
-- self.hparams.voc_size = 100000
-- self.hparams.train_loop_check_freq = 10
-- self.hparams.dropout_rnn = 0.5
-- self.hparams.dropout_fc = 0.5
-- self.hparams.batch_size = 256
-- self.hparams.fc_hidden_size = 100
-- self.hparams.rnn_hidden_size = 50
-- self.hparams.cnn_hidden_size = 200
-- self.hparams.cnn_kernal_size = 3
-- self.hparams.rnn_num_layers = 1
-- self.hparams.check_early_stop = True
-- self.hparams.es_look_back = 50
-- self.hparams.no_imp_look_back = 25
-- self.hparams.decay_lr_no_improv = 0.5
-- self.hparams.es_req_prog = 0.0
-- self.hparams.optim_enc = <class 'torch.optim.adam.Adam'>
-- self.hparams.optim_dec = <class 'torch.optim.adam.Adam'>
-- self.hparams.scheduler = <class 'torch.optim.lr_scheduler.ExponentialLR'>
-- self.hparams.scheduler_gamma = 0.95
-- self.hparams.criterion = 

In [9]:
mgr.model.eval_model(mgr.dataloader.loaders['val'])

(70.3, 3.550516128540039)

### On the MNLI Set:

In [10]:
cnn_res = {}
for genre in conf.GENRE_LIST:
    mgr.load_data(mm.loaderRegister.MNLI, genre=genre)
    acc = mgr.model.eval_model(mgr.dataloader.loaders['val'])[0]
    cnn_res[genre] = acc
    
cnn_res

[2018-10-28 13:06:47] [INFO] Loading data using MNLI ...
[2018-10-28 13:06:47] [INFO] loading raw training data set ...
[2018-10-28 13:06:47] [INFO] loading raw training data set ...
[2018-10-28 13:06:47] [INFO] loading pre-trained word vectors, building vocab ...
[2018-10-28 13:07:00] [INFO] converting training set to index ...
[2018-10-28 13:07:00] [INFO] converting val set to index ...
[2018-10-28 13:07:00] [INFO] piping data into pytorch DataLoaders ...
[2018-10-28 13:07:00] [INFO] Loading data using MNLI ...
[2018-10-28 13:07:00] [INFO] loading raw training data set ...
[2018-10-28 13:07:01] [INFO] loading raw training data set ...
[2018-10-28 13:07:01] [INFO] loading pre-trained word vectors, building vocab ...
[2018-10-28 13:07:14] [INFO] converting training set to index ...
[2018-10-28 13:07:14] [INFO] converting val set to index ...
[2018-10-28 13:07:14] [INFO] piping data into pytorch DataLoaders ...
[2018-10-28 13:07:14] [INFO] Loading data using MNLI ...
[2018-10-28 13:07:1

{'fiction': 45.82074521651561,
 'telephone': 46.46766169154229,
 'slate': 43.11377245508982,
 'government': 43.503937007874015,
 'travel': 46.537678207739305}